In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as const
import astropy.units as u
import Corrfunc
from Corrfunc.theory.xi import xi
from Corrfunc.theory.DD import DD
from itertools import product

In [2]:
n_threads = 8
dtype = np.float64
N_min = 1000
sf_cut = 1e-10
N_bins = 50
r_edges = np.linspace(1, 20, N_bins+1)
r_avg = (3/4) * (r_edges[1:]**4 - r_edges[:-1]**4) / (r_edges[1:]**3 - r_edges[:-1]**3)
V_shell = (4/3) * np.pi * (r_edges[1:]**3 - r_edges[:-1]**3)
Edd_coeff = ((4 * np.pi * const.G * const.m_p * u.Msun) / (0.1 * const.sigma_T * const.c)).to(u.Msun/u.yr).value

N_side = 3
N_cells = N_side**3

L_box_TNG = dtype(205) #Mpc/h
L_cell_TNG = L_box_TNG/N_side
V_ratio_TNG = V_shell / (L_box_TNG**3 - L_cell_TNG**3)

L_box_ASTRID = dtype(250) #Mpc/h
L_cell_ASTRID = L_box_ASTRID/N_side
V_ratio_ASTRID = V_shell / (L_box_ASTRID**3 - L_cell_ASTRID**3)

def load(sim, z, bhmass_min, fEdd_min, stmass_min, sf):
    if sim=='TNG':
        L_box = L_box_TNG
        L_cell = L_cell_TNG
        V_ratio = V_ratio_TNG
        bhmasses_5 = np.load(f'bhmasses-5_TNG300_z{z}.npy')
        bhars_5 = np.load(f'bhars-5_TNG300_z{z}.npy')
        positions_5 = np.load(f'positions-5_TNG300_z{z}.npy')
        stmasses_sub = np.load(f'stmasses-sub_TNG300_z{z}.npy')
        sfrs_sub = np.load(f'sfrs-sub_TNG300_z{z}.npy')
        positions_sub = np.load(f'positions-sub_TNG300_z{z}.npy')
    else: #ASTRID
        L_box = L_box_ASTRID
        L_cell = L_cell_ASTRID
        V_ratio = V_ratio_ASTRID
        if z==1:
            snap = 544
        elif z==2:
            snap = 348
        else:
            return None
        bhmasses_5 = np.load(f'bhmasses_{snap}_5.npy')
        bhars_5 = np.load(f'bhars_{snap}_5.npy')
        positions_5 = np.load(f'positions_{snap}_5.npy')
        stmasses_sub = np.load(f'stmasses-2r_{snap}_sub.npy')
        sfrs_sub = np.load(f'sfrs-2r_{snap}_sub.npy')
        positions_sub = np.load(f'positions_{snap}_sub.npy')
    fEdd_5 = bhars_5 / (Edd_coeff * bhmasses_5)

    if fEdd_min == -1:
        mask_reqs_q = (bhmasses_5 > bhmass_min)
    else:
        mask_reqs_q = (bhmasses_5 > bhmass_min) & (fEdd_5 > fEdd_min)
    if sf == True:
        mask_reqs_g = (stmasses_sub > stmass_min) & (sfrs_sub/stmasses_sub >= sf_cut)
    else:
        mask_reqs_g = (stmasses_sub > stmass_min) & (sfrs_sub/stmasses_sub < sf_cut)

    if np.sum(mask_reqs_q) < N_min:
        return None
    elif np.sum(mask_reqs_g) < N_min:
        return None

    #Mpc/h
    Xq = np.ascontiguousarray(positions_5[mask_reqs_q][:,0] / 1000, dtype)
    Yq = np.ascontiguousarray(positions_5[mask_reqs_q][:,1] / 1000, dtype)
    Zq = np.ascontiguousarray(positions_5[mask_reqs_q][:,2] / 1000, dtype)
    Xg = np.ascontiguousarray(positions_sub[mask_reqs_g][:,0] / 1000, dtype)
    Yg = np.ascontiguousarray(positions_sub[mask_reqs_g][:,1] / 1000, dtype)
    Zg = np.ascontiguousarray(positions_sub[mask_reqs_g][:,2] / 1000, dtype)
    
    cell_inds_xq = np.floor(Xq / L_cell)
    cell_inds_yq = np.floor(Yq / L_cell)
    cell_inds_zq = np.floor(Zq / L_cell)
    cell_inds_xg = np.floor(Xg / L_cell)
    cell_inds_yg = np.floor(Yg / L_cell)
    cell_inds_zg = np.floor(Zg / L_cell)
    
    c = 0
    xiqq_mat = np.zeros((N_cells, N_bins), dtype=dtype)
    xigg_mat = np.zeros((N_cells, N_bins), dtype=dtype)
    xiqg_mat = np.zeros((N_cells, N_bins), dtype=dtype)
    
    for i in range(N_side):
        for j in range(N_side):
            for k in range(N_side):
                mask_keep_q = ~((cell_inds_xq == i) & (cell_inds_yq == j) & (cell_inds_zq == k))
                mask_keep_g = ~((cell_inds_xg == i) & (cell_inds_yg == j) & (cell_inds_zg == k))

                results_qq = DD(autocorr=True, nthreads=n_threads, binfile=r_edges, \
                                X1=Xq[mask_keep_q], Y1=Yq[mask_keep_q], Z1=Zq[mask_keep_q], \
                                periodic=True, boxsize=L_box)
                results_gg = DD(autocorr=True, nthreads=n_threads, binfile=r_edges, \
                                X1=Xg[mask_keep_g], Y1=Yg[mask_keep_g], Z1=Zg[mask_keep_g], \
                                periodic=True, boxsize=L_box)
                results_qg = DD(autocorr=False, nthreads=n_threads, binfile=r_edges, \
                                X1=Xq[mask_keep_q], Y1=Yq[mask_keep_q], Z1=Zq[mask_keep_q], \
                                X2=Xg[mask_keep_g], Y2=Yg[mask_keep_g], Z2=Zg[mask_keep_g], \
                                periodic=True, boxsize=L_box)

                DDqq = results_qq['npairs'] #Double-counted
                DDgg = results_gg['npairs'] #Double-counted
                DDqg = results_qg['npairs'] #Single-counted

                Nq_keep = np.sum(mask_keep_q)
                Ng_keep = np.sum(mask_keep_g)
                RRqq = Nq_keep * (Nq_keep - 1) * V_ratio #Double-counted
                RRgg = Ng_keep * (Ng_keep - 1) * V_ratio #Double-counted
                RRqg = Nq_keep * Ng_keep * V_ratio #Single-counted

                xiqq = DDqq/RRqq - 1
                xigg = DDgg/RRgg - 1
                xiqg = DDqg/RRqg - 1
    
                xiqq_mat[c,:] = xiqq
                xigg_mat[c,:] = xigg
                xiqg_mat[c,:] = xiqg
                c += 1
                
    if len(Xg) > len(Xq):
        R_mat = xiqg_mat / xigg_mat #cross
    else:
        R_mat = np.sqrt(xiqq_mat / xigg_mat) #auto
    
    
    xiqq_mean_arr = np.mean(xiqq_mat, axis=0)
    xigg_mean_arr = np.mean(xigg_mat, axis=0)
    xiqg_mean_arr = np.mean(xiqg_mat, axis=0)
    R_mean_arr = np.mean(R_mat, axis=0)
    
    xiqq_sigma_arr = np.sqrt(np.sum((xiqq_mat - xiqq_mean_arr)**2, axis=0) * (N_cells-1) / N_cells)
    xigg_sigma_arr = np.sqrt(np.sum((xigg_mat - xigg_mean_arr)**2, axis=0) * (N_cells-1) / N_cells)
    xiqg_sigma_arr = np.sqrt(np.sum((xiqg_mat - xiqg_mean_arr)**2, axis=0) * (N_cells-1) / N_cells)
    R_sigma_arr = np.sqrt(np.sum((R_mat - R_mean_arr)**2, axis=0) * (N_cells-1) / N_cells)

    mask = np.isfinite(R_mean_arr) & np.isfinite(R_sigma_arr) & (R_sigma_arr > 0)
    if not np.any(mask):
        return None
    try:
        constant, cov = np.polyfit(r_avg[mask], R_mean_arr[mask], 0, w=1/R_sigma_arr[mask], cov=True)
    except Exception:
        return None
    constant_err = np.sqrt(cov)

    return xiqq_mean_arr, xigg_mean_arr, xiqg_mean_arr, R_mean_arr, xiqq_sigma_arr, xigg_sigma_arr, xiqg_sigma_arr, R_sigma_arr, constant[0], constant_err[0][0]

In [3]:
sim_list = ['TNG', 'ASTRID']
z_list = [1, 2, 3]
bhmass_min_list = [10**(7), 10**(7.5), 10**(8), 10**(8.5), 10**(9)]
fEdd_min_list = [-1, 0.001, 0.01, 0.05, 0.1]
stmass_min_list = [10**(10), 10**(10.25), 10**(10.5), 10**(10.75), 10**(11)]
sf_list = [True, False]
combos = list(product(sim_list, z_list, bhmass_min_list, fEdd_min_list, stmass_min_list, sf_list))

In [4]:
keep_sim_list = []
keep_z_list = []
keep_bhmass_min_list = []
keep_fEdd_min_list = []
keep_stmass_min_list = []
keep_sf_list = []

xiqq_mean_arr_list = []
xigg_mean_arr_list = []
xiqg_mean_arr_list = []
R_mean_arr_list = []
xiqq_sigma_arr_list = []
xigg_sigma_arr_list = []
xiqg_sigma_arr_list = []
R_sigma_arr_list = []

constant_list = []
constant_err_list = []

for combo in combos:
    sim, z, bhmass_min, fEdd_min, stmass_min, sf = combo
    result = load(sim, z, bhmass_min, fEdd_min, stmass_min, sf)

    if not result==None:
        xiqq_mean_arr, xigg_mean_arr, xiqg_mean_arr, R_mean_arr, \
            xiqq_sigma_arr, xigg_sigma_arr, xiqg_sigma_arr, R_sigma_arr, \
            constant, constant_err = result

        keep_sim_list.append(sim)
        keep_z_list.append(z)
        keep_bhmass_min_list.append(bhmass_min)
        keep_fEdd_min_list.append(fEdd_min)
        keep_stmass_min_list.append(stmass_min)
        keep_sf_list.append(sf)
        
        xiqq_mean_arr_list.append(xiqq_mean_arr)
        xigg_mean_arr_list.append(xigg_mean_arr)
        xiqg_mean_arr_list.append(xiqg_mean_arr)
        R_mean_arr_list.append(R_mean_arr)
        xiqq_sigma_arr_list.append(xiqq_sigma_arr)
        xigg_sigma_arr_list.append(xigg_sigma_arr)
        xiqg_sigma_arr_list.append(xiqg_sigma_arr)
        R_sigma_arr_list.append(R_sigma_arr)

        constant_list.append(constant)
        constant_err_list.append(constant_err)

/var/folders/8y/zr0r45091_l1470gh3n3j5bw0000gn/T/ipykernel_3727/4088113683.py:124: RuntimeWarning: invalid value encountered in sqrt
  R_mat = np.sqrt(xiqq_mat / xigg_mat) #auto
/var/folders/8y/zr0r45091_l1470gh3n3j5bw0000gn/T/ipykernel_3727/4088113683.py:56: RuntimeWarning: invalid value encountered in divide
  mask_reqs_g = (stmasses_sub > stmass_min) & (sfrs_sub/stmasses_sub >= sf_cut)
/var/folders/8y/zr0r45091_l1470gh3n3j5bw0000gn/T/ipykernel_3727/4088113683.py:58: RuntimeWarning: invalid value encountered in divide
  mask_reqs_g = (stmasses_sub > stmass_min) & (sfrs_sub/stmasses_sub < sf_cut)


In [5]:
np.savez_compressed('Corr_combos_data.npz', sim=keep_sim_list, z=keep_z_list, \
                    bhmass_min=keep_bhmass_min_list, fEdd_min=keep_fEdd_min_list, stmass_min=keep_stmass_min_list, sf=keep_sf_list, \
                    xiqq=xiqq_mean_arr_list, xigg=xigg_mean_arr_list, xiqg=xiqg_mean_arr_list, R=R_mean_arr_list, \
                    xiqq_err=xiqq_sigma_arr_list, xigg_err=xigg_sigma_arr_list, xiqg_err=xiqg_sigma_arr_list, R_err=R_sigma_arr_list, \
                    constant=constant_list, constant_err=constant_err_list)